In [56]:
import numpy as np
import datetime

This code generates g-code files for the CNC router to run. The router holds a robot tool and raster scans to generate desired coverages on a test surface.

In [57]:
# USER SETTINGS
# desired_coverages = np.array([0.005, 0.01, 0.02, 0.04, 0.08]) # Proportion of each square covered in dimpled area (should be < 1), [unitless]
desired_coverages = np.array([0.32]) # Proportion of each square covered in dimpled area (should be < 1), [unitless]
dimple_diameter = 0.00669 # Diameter of applied dimples, [in]
dimple_rate = 3.9         # Rate of dimple application, [Hz]
square_side = 0.5         # Length of one side of the square area to be textured, [in]
rapid_height = 1.0        # Safe height above the surface, [in]
feed_height = 0.1         # Height above the surface when feeding must start, [in]

In [58]:
num_squares = desired_coverages.shape[0]        # Number of squares the user wants, [unitless]
square_area = square_side**2                    # Area of eeach square, [in^2]
dimple_area = 3.1415 * dimple_diameter**2 / 4   # Area of each dimple, [in^2]
coverage_area = square_area * desired_coverages # Area the dimples should take up in each square, [in^2]
num_dimples = coverage_area / dimple_area       # Total number of dimples in each square, [unitless]
dimples_per_row = num_dimples**0.5              # Each row should have this many dimples, [unitless]
dx = square_side / dimples_per_row              # Distance between successive dimples, [in]
dt = 1 / dimple_rate                            # Time between successive dimples, [s]
time_per_square = num_dimples / dimple_rate     # Estimated time required to achieve the desired coverage in each square, [s]
feedrates = (dx / dt) * 60                      # Feedrate for CNC linear moves, [in/minute]
total_time = np.sum(time_per_square)            # Total time to make the calibration squares

print("Design output for {} squares. Per-square statistics are:".format(int(num_squares)))
for i in range(num_squares):
    print("Square {},".format(i), end="\t")
    print("Coverage {:5.2f}%,".format(desired_coverages[i]*100), end="\t")
    print("Dimples {:.1f},".format(num_dimples[i]),end="\t")
    print("Duration {} [d:m:s],".format(datetime.timedelta(seconds=int(time_per_square[i]))), end="\t")
    print("Feedrate {:5.2f} [in/min]".format(feedrates[i]), end="\t")
    print()
print("Total time for calibration square production: {} [d:m:s]".format(datetime.timedelta(seconds=int(total_time))))

Design output for 1 squares. Per-square statistics are:
Square 0,	Coverage 32.00%,	Dimples 2275.9,	Duration 0:09:43 [d:m:s],	Feedrate  2.45 [in/min]	
Total time for calibration square production: 0:09:43 [d:m:s]


Copy-paste G-Codes:
```
"G00\n" + # rapid move
"G01\n" + # feed move
"G17\n" + # XY plane selection
"G20\n" + # inches unit system
"G21\n" + # millimeters unit system
"G28\n" + # rapid zero return
"G54\n" + # select G54 work coordinate system
"G90\n" + # absolute mode
"G91\n" + # incremental mode
"G94\n" + # feed per minute mode
```

In [59]:
# Create a descriptive name of the file
path = "Output/"
ending = ""
for i in range(num_squares): 
    ending = ending + str(desired_coverages[i]*100) + "_"
ending = ending.replace(".","p")
filename = "calibration_" + ending + ".ncd"

# The preamble is the set of general setup lines at the beginning of the program
preamble = ("G94\n" + # feed per minute mode
            "G17\n" + # XY plane selection
            "G20\n" + # inches unit system
            "G90\n" + # absolute mode
            "G54\n" + # select G54 work coordinate system
            "G28\n" + # rapid zero return
            "\n")

# The first moves run after the preamble and position the tool appropriately to start the main moves
first_moves = ("G00 X0.0 Y0.0\n" + # Move to the origin (bottom left corner of the first square)
               "Z-{:.3f}\n".format(rapid_height - feed_height)       + # Rapid move down to the feed height
               "G01 Z-{:.3f} F5.0\n".format(rapid_height)            + # Set the correct feedrate and move to Z zero
               "\n")

# The main moves are doing the actual dimpling
y = 0.0 
current_square = 0
main_moves = ""
direction = True 

while (current_square < num_squares):
    feedrate = feedrates[current_square]
    if direction: # Moving left to right
        main_moves = main_moves + "X{:.3f} F{:.3f}\n".format(square_side,feedrate)
        direction = False
    else: # Moving right to left
        main_moves = main_moves + "X0.0 F{:.3f}\n".format(feedrate)
        direction = True
    y = y + dx[current_square]
    main_moves = main_moves + "Y{:.3f} F{:.3f}\n".format(y,feedrate)
    current_square = np.floor(y / square_side).astype(int)
main_moves = main_moves + "\n"

# The last moves pick the tool up and return it to the correct starting location
last_moves = ("G00 Z0.0\n" + # Rapid move the tool up to clear the workpiece
              "X0.0 Y0.0")   # Return to origin

In [60]:
with open(path+filename,"w") as file:
    file.write(preamble)
    file.write(first_moves)
    file.write(main_moves)
    file.write(last_moves)